# E.16. 프로젝트 - Movielens 영화 SBR
##  Movielens 1M Dataset을 기반으로, Session based Recommendation 시스템 제작

### 평가기준
1. Movielens 데이터셋을 session based recommendation 관점으로 전처리하는 과정이 체계적으로 진행되었다.<br>
- 데이터셋의 면밀한 분석을 토대로 세션단위 정의 과정(길이분석, 시간분석)을 합리적으로 수행한 과정이 기술되었다.<br>
=> 하였음.
2. RNN 기반의 예측 모델이 정상적으로 구성되어 안정적으로 훈련이 진행되었다. <br>
- 적절한 epoch만큼의 학습이 진행되는 과정에서 train loss가 안정적으로 감소하고, validation 단계에서의 Recall, MRR이 개선되는 것이 확인된다.<br>
=> 하였음. (안정적이거나 개선되는 것은 논외로 함)
3. 세션정의, 모델구조, 하이퍼파라미터 등을 변경해서 실험하여 Recall, MRR 등의 변화추이를 관찰하였다.<br>
- 3가지 이상의 변화를 시도하고 그 실험결과를 체계적으로 분석하였다.<br>
=> 최대한 체계적으로 분석함.


### 마치며
session의 길이를 session id 에서 user id로 바꿔서 진행함.
전처리를 한 후 학습 후 생각과는 다르게 ReCall@20, MRR@20은 결과가 좋지 않음. 
점점 어려워 지는 것 같다. 

In [12]:
import pandas as pd
import tensorflow as tf
import numpy as np
from datetime import datetime as dt
import datetime

import os
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

print(pd.__version__)
print(tf.__version__)

1.3.3
2.6.0


In [13]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], 
                       dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data['Time'] = data['Time'].apply(lambda t: dt.fromtimestamp(t)) # Timestamp 타입을 datetime으로 변경 함.
# data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data.sort_values(['Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data



,UserId,ItemId,Rating,Time
1000138,6040,858,4,2000-04-25 23:05:32
1000153,6040,2384,4,2000-04-25 23:05:54
999873,6040,593,5,2000-04-25 23:05:54
1000007,6040,1961,4,2000-04-25 23:06:17
1000192,6040,2019,5,2000-04-25 23:06:17
...,...,...,...,...
825793,4958,2399,1,2003-02-28 17:45:38
825438,4958,1407,5,2003-02-28 17:47:23
825724,4958,3264,4,2003-02-28 17:49:08
825731,4958,2634,3,2003-02-28 17:49:08


## Step 1. 데이터의 전처리

In [14]:
data['UserId'].nunique(), data['ItemId'].nunique()

(6040, 3706)

In [15]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

# 대략 3년치의 자료를 가지고 있다.

2000-04-25 23:05:32
2003-02-28 17:49:50


In [16]:
month_ago = latest - datetime.timedelta(30*2)  # 최종 날짜로부터 30 * 2 일 이전 날짜를 구한다.  
data = data[data['Time'] > month_ago]   # 방금 구한 날짜 이후의 데이터만 모은다. 
data

,UserId,ItemId,Rating,Time
497017,3054,1269,4,2002-12-30 18:21:02
497225,3054,2133,2,2002-12-30 18:22:28
158293,1015,1375,3,2002-12-30 19:06:03
157589,1015,3791,3,2002-12-30 19:07:52
76351,518,2881,4,2002-12-30 19:37:46
...,...,...,...,...
825793,4958,2399,1,2003-02-28 17:45:38
825438,4958,1407,5,2003-02-28 17:47:23
825724,4958,3264,4,2003-02-28 17:49:08
825731,4958,2634,3,2003-02-28 17:49:08


In [17]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

2002-12-30 18:21:02
2003-02-28 17:49:50


### 데이터 Cleansing

In [18]:
# UserId별 시간(초)별 중복이 있는 데이터는 삭제하도록 한다.
# - 한 사용자가 같은 시간에 여러개의 데이터를 입력한 경우는 뭔가 비정상적인 작동일 확률이 높다. 
data = data.drop_duplicates(subset=['UserId', 'Time'], keep='first')
data

,UserId,ItemId,Rating,Time
497017,3054,1269,4,2002-12-30 18:21:02
497225,3054,2133,2,2002-12-30 18:22:28
158293,1015,1375,3,2002-12-30 19:06:03
157589,1015,3791,3,2002-12-30 19:07:52
76351,518,2881,4,2002-12-30 19:37:46
...,...,...,...,...
825526,4958,3489,4,2003-02-28 17:45:20
825793,4958,2399,1,2003-02-28 17:45:38
825438,4958,1407,5,2003-02-28 17:47:23
825724,4958,3264,4,2003-02-28 17:49:08


In [19]:
# Rating이 낮은 경우 의 추천도 사용 합니다. 
# - session click의 경우와 달리 사용자 아이디별 등수가 입력된 것이므로 의미 있다고 생각됨. 

# 인기가 없는 item도 사용하도록 한다. 
# - session click의 경우 의미없이(사용자의 의지와는 별도로) 생성된 데이터인 경우를 삭제 하는 것인데. 
#   이번 경우는 그럴 경우가 없을 것 같다. 은 삭제하도록 한다. 5이하의 클릭을 받은 item은 제거한다.

### Train/ Valid/ Test split

In [20]:
# Time별로 정렬한 후 Split한다.

data.sort_values(by=['Time'], inplace=True) 
data.shape

(2323, 4)

In [21]:
# split_by_date() 함수를 사용해 마지막의 3일씩을 각각 test, val로 split

def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    session_last_time = data.groupby('UserId')['Time'].max()
    session_in_train = session_last_time[session_last_time < final_time - datetime.timedelta(n_days)].index
    session_in_test = session_last_time[session_last_time >= final_time - datetime.timedelta(n_days)].index

    before_date = data[data['UserId'].isin(session_in_train)]
    after_date = data[data['UserId'].isin(session_in_test)]
    after_date = after_date[after_date['ItemId'].isin(before_date['ItemId'])]
    return before_date, after_date

tr, test = split_by_date(data, n_days=3)
tr, val = split_by_date(tr, n_days=3)

In [23]:
# data에 대한 정보를 살펴봅니다.
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Users: {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')    
    
stats_info(tr, 'Train')
stats_info(val, 'Valid')
stats_info(test, 'Test')

* Train Set Stats Info
	 Events: 1687
	 Users: 151
	 Items: 1057
	 First Time : 2002-12-30 18:21:02
	 Last Time : 2003-02-22 06:41:03

* Valid Set Stats Info
	 Events: 84
	 Users: 13
	 Items: 79
	 First Time : 2002-12-31 04:06:25
	 Last Time : 2003-02-24 19:35:37

* Test Set Stats Info
	 Events: 281
	 Users: 17
	 Items: 247
	 First Time : 2002-12-31 14:25:55
	 Last Time : 2003-02-28 17:45:20



In [24]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [25]:
save_path = data_path / 'processed_pj'
save_path.mkdir(parents=True, exist_ok=True)

print(save_path)

tr.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

/aiffel/aiffel/yoochoose/data/processed_pj


### 미니 배치의 구성

In [26]:
# 전처리한 데이터셋 불러오기

import pickle
from pathlib import Path
import os

data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/processed_pj') 
train_path = data_path / 'train.pkl'
val_path = data_path / 'valid.pkl'
test_path = data_path / 'test.pkl'

# load
with open(train_path, 'rb') as f:
    tr = pickle.load(f)
    
# load
with open(val_path, 'rb') as f:
    val = pickle.load(f)
    
# load
with open(test_path, 'rb') as f:
    test = pickle.load(f)    

In [27]:
# UserId로 offsets
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

In [28]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

### 모델 구성

In [29]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [31]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [32]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

### 모델 학습

In [37]:
# SessionId -> UserId
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['UserId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['UserId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [38]:
args = Args(tr, val, test, batch_size=50, hsz=50, drop_rate=0.1, lr=0.001, epochs=10, k=20)
args = Args(tr, val, test, batch_size=50, hsz=50, drop_rate=0.2, lr=0.002, epochs=20, k=20)
args = Args(tr, val, test, batch_size=64, hsz=50, drop_rate=0.2, lr=0.001, epochs=15, k=20)
args = Args(tr, val, test, batch_size=50, hsz=50, drop_rate=0.1, lr=0.001, epochs=15, k=20)


In [39]:
model = create_model(args)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(50, 1, 1057)]           0         
_________________________________________________________________
GRU (GRU)                    [(50, 50), (50, 50)]      166350    
_________________________________________________________________
dropout_1 (Dropout)          (50, 50)                  0         
_________________________________________________________________
dense_1 (Dense)              (50, 1057)                53907     
Total params: 220,257
Trainable params: 220,257
Non-trainable params: 0
_________________________________________________________________


In [40]:
train_model(model, args)

Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]


IndexError: index 13 is out of bounds for axis 0 with size 13

### 모델 테스트

In [ ]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(tr, model, args, 3)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)
